In [1]:
import bottlenecking as btl
import tensorflow as tf
import numpy as np
import pandas as pd
import uns
from uns import training
from fcn_model import fcn16_vgg

In [2]:
sess = tf.InteractiveSession()

In [3]:
batch_images = tf.placeholder(tf.float32,[None, 420, 580, 3])
#batch_masks = tf.placeholder(tf.float32, [None, 420, 580, 2])  #Size 2 => "yes" and "no"

In [4]:
vgg_path = '../tensorflow-fcn/vgg16.npy'

In [5]:
net = fcn16_vgg.FCN16VGG(vgg16_npy_path=vgg_path)

npy file loaded


In [6]:
with tf.name_scope("content_vgg"):
       net.build(batch_images, train=False, num_classes=2, random_init_fc8=True,debug=False)

Layer name: conv1_1
Layer shape: (3, 3, 3, 64)
Layer name: conv1_2
Layer shape: (3, 3, 64, 64)
Layer name: conv2_1
Layer shape: (3, 3, 64, 128)
Layer name: conv2_2
Layer shape: (3, 3, 128, 128)
Layer name: conv3_1
Layer shape: (3, 3, 128, 256)
Layer name: conv3_2
Layer shape: (3, 3, 256, 256)
Layer name: conv3_3
Layer shape: (3, 3, 256, 256)
Layer name: conv4_1
Layer shape: (3, 3, 256, 512)
Layer name: conv4_2
Layer shape: (3, 3, 512, 512)
Layer name: conv4_3
Layer shape: (3, 3, 512, 512)
Layer name: conv5_1
Layer shape: (3, 3, 512, 512)
Layer name: conv5_2
Layer shape: (3, 3, 512, 512)
Layer name: conv5_3
Layer shape: (3, 3, 512, 512)
Layer name: fc6
Layer shape: [7, 7, 512, 4096]
Layer name: fc7
Layer shape: [1, 1, 4096, 4096]


In [7]:
bottleneck=[net.pool4, net.fc6]

In [8]:
btl.get_bottleneck_dims(bottleneck[0])

[27, 37, 512]

In [15]:
def build_batch_output_w_masks(imgs,masks,path,bottlenecks,sess):
    """writes a single batch to disk"""
    batch_size = imgs.shape[0]
    mask_dims=masks.shape[1:]
    mask_flat_len=mask_dims[0]*mask_dims[1]*mask_dims[2]
    fc6_dims = btl.get_bottleneck_dims(bottlenecks[0])
    fc6_flat_len = fc6_dims[0]*fc6_dims[1]*fc6_dims[2]
    pool_dims = btl.get_bottleneck_dims(bottlenecks[1])
    pool_flat_len=pool_dims[0]*pool_dims[1]*pool_dims[2]
    bottle_fc6, bottle_pool = sess.run(bottlenecks, feed_dict={batch_images:imgs})
    flat_fc6 = bottle_fc6.reshape(-1,fc6_flat_len)
    flat_masks = masks.reshape(-1,mask_flat_len)
    flat_pool = bottle_pool.reshape(-1,pool_flat_len)
    print(flat_fc6.shape, flat_pool.shape,flat_masks.shape)

    records = np.concatenate((flat_fc6,flat_pool,masks),axis=1)
    print(records.shape)
    print('Finished running forward passes')
    records.astype('float32').tofile(path)
    msg = 'Output data written to {file}'.format(file=path)
    print(msg)


def build_output_data(session,bottlenecks,batch_size,trainingData,target_dir,name='fc6',debug=False):    
    """write a bunch of batches to disk"""
    batches = btl.make_batches(trainingData,batch_size)
    if debug == True:
        batches = batches[:4]
    for i,b in enumerate(batches):
        #images = btl.load_batch(b)
        images, masks = btl.load_batch(b)
        filename = target_dir+name +'_batch_' + str(i+1)
        #build_batch_output(images,filename,bottleneck,session)
        build_batch_output_w_masks(images, masks,filename,bottlenecks,session)

In [10]:
### test making first 12 training images into 4 batches of 3 images

In [11]:
time sess.run(tf.initialize_all_variables())

CPU times: user 3.87 s, sys: 6.15 s, total: 10 s
Wall time: 21.7 s


In [16]:
build_output_data(sess,[net.fc6,net.pool4],4,training[:4],'/Users/gus/CDIPS')

(4, 1089536) (4, 511488) (4, 487200)


ValueError: all the input arrays must have same number of dimensions